In [17]:
import konlpy
import json
import urllib.request
import requests
import re
from googletrans import Translator

In [18]:
def ends_with_jongsung(letter): # 받침 유무 확인하기 받침이 있으면 True, 없으면 False 반환
    m = re.search(r"[가-힣|ㄴ]?", letter)
    if m:
        n = m.group()[-1]
        return (ord(n)-ord("가")) % 28 > 0 # 나머지가 있으면 받침 있음. 반환값 생김.
    else:
        return None

In [19]:
def josa_change(letter, letter2): # 주격 조사 알맞게 change
    josa = ''
    if letter2 == "을" or letter2 == "를":
        josa = "을" if ends_with_jongsung(letter) else "를" # 앞에 받침 있는 경우, 뒤에는 받침이 없는 경우
    elif letter2 == "이" or letter2 == "가":
        josa = "이" if ends_with_jongsung(letter) else "가"
    elif letter2 == "과" or letter2 == "와":
        josa = "과" if ends_with_jongsung(letter) else "와"
    elif letter2 == "은" or letter2 == "는":
        josa = "은" if ends_with_jongsung(letter) else "는"

    return josa

In [26]:
## PaPago로 보그체 만들기
def Papago_Vogulize(Toknized_Korean_Okt):
    valid_grammer_Okt = ['Noun', 'Exlamation', 'Adjective']


    client_id = "DItIw9wyPPPRDPx4Hy46"
    client_secret = "L6ZEleX9Mp"
    url = "https://openapi.naver.com/v1/papago/n2mt"
    url2 = "https://transliterator.herokuapp.com/"
    sess = requests.Session()

    vogulized_sentence = ''


    for word in range(len(Toknized_Korean_Okt)):
        for w_divided in range(len(Toknized_Korean_Okt[word])):
                if (Toknized_Korean_Okt[word][w_divided][1] in valid_grammer_Okt) and (len(Toknized_Korean_Okt[word][w_divided][0]) > 1):
                    # print("번역 가능한 품사 단어 : " + Toknized_Korean_Okt[word][w_divided][0])
                    Toknized_Text_English = urllib.parse.quote(Toknized_Korean_Okt[word][w_divided][0])
                    data = "source=ko&target=en&text=" + Toknized_Text_English
                    request = urllib.request.Request(url)
                    request.add_header("X-Naver-Client-Id", client_id)
                    request.add_header("X-Naver-Client-Secret", client_secret)
                    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
                    rescode = response.getcode()

                    if rescode==200:
                        response_body = response.read()
                        json_word = json.loads(response_body.decode("utf-8"))
                        result = json_word['message']['result']['translatedText']
                        Toknized_Korean_Okt[word][w_divided][0]

                        payload = {'input': result}
                        res = sess.post(url2, data=payload)
                        konglish = res.json()['output'].replace('?','')
                        Toknized_Korean_Okt[word][w_divided][0] = konglish
                        # print(konglish)
                        result = re.sub(r'\.', '', result)
                        konglish = str(konglish + '(' + result + ')')
                        vogulized_sentence += konglish

                    else:
                        print("Error")

                else:
                    # print("번역 불가능한 품사 단어 : " + Toknized_Korean_Okt[word][w_divided][0])

                    if Toknized_Korean_Okt[word][w_divided][1] == 'Josa' and Toknized_Korean_Okt[word][w_divided][0] in ['은', '는', '이', '가', '과', '와', '을', '를']:
                        Toknized_Korean_Okt[word][w_divided][0] = josa_change(Toknized_Korean_Okt[word][w_divided-1][0][-1], Toknized_Korean_Okt[word][w_divided][0])

                    vogulized_sentence += str(Toknized_Korean_Okt[word][w_divided][0])

        vogulized_sentence += ' '

    print()
    print(vogulized_sentence)
    print()
    print(Korean_Text_b)

In [27]:
## 구글 번역기 보그체로 바꾸기
def Google_Vogulize(Toknized_Korean_Okt):
    valid_grammer_Okt = ['Noun', 'Exlamation', 'Adjective']


    translator = Translator(service_urls=['translate.google.co.kr'])
    url2 = "https://transliterator.herokuapp.com/"
    sess = requests.Session()
    vogulized_sentence = ''

    for word in range(len(Toknized_Korean_Okt)):
        for w_divided in range(len(Toknized_Korean_Okt[word])):
                if Toknized_Korean_Okt[word][w_divided][1] in valid_grammer_Okt and (len(Toknized_Korean_Okt[word][w_divided][0]) > 1):
                    # print("번역 가능한 품사 단어 : " + Toknized_Korean_Okt[word][w_divided][0])

                    konglish = translator.translate(Toknized_Korean_Okt[word][w_divided][0], src='ko', dest='en').text
                    result2 = konglish
                    payload = {'input': konglish}
                    res = sess.post(url2, data=payload)
                    konglish = res.json()['output'].replace('?','')
                    Toknized_Korean_Okt[word][w_divided][0] = konglish
                    konglish = str(konglish + '(' + result2 + ')')
                    # print(konglish)

                    vogulized_sentence += konglish

                else:
                    # print("번역 불가능한 품사 단어 : " + Toknized_Korean_Okt[word][w_divided][0])

                    if Toknized_Korean_Okt[word][w_divided][1] == 'Josa' and Toknized_Korean_Okt[word][w_divided][0] in ['은', '는', '이', '가', '과', '와', '을', '를']:
                        Toknized_Korean_Okt[word][w_divided][0] = josa_change(Toknized_Korean_Okt[word][w_divided-1][0][-1], Toknized_Korean_Okt[word][w_divided][0])

                    vogulized_sentence += str(Toknized_Korean_Okt[word][w_divided][0])

        vogulized_sentence += ' '

    print()
    print(vogulized_sentence)
    print()
    print(Korean_Text_b)

In [29]:
if __name__ == "__main__":

    print("보그체로 변환하고 싶은 문장을 입력하세요.")
    Korean_Text_b = input()
    Korean_Text = Korean_Text_b.split(sep=' ')

    Toknize_Okt = konlpy.tag.Okt()
    Toknized_Korean_Okt = []


    for i in range(len(Korean_Text)):
        Toknized_Korean_Okt.append(Toknize_Okt.pos(Korean_Text[i]))

        for j in range(len(Toknized_Korean_Okt[i])):
            Toknized_Korean_Okt[i][j] = list(Toknized_Korean_Okt[i][j])

    print("보그체로 변환할 때 사용할 번역기의 숫자를 입력하세요. (1: 네이버 파파고, 2: 구글 번역기)")
    num = int(input())
    while num != 1 or num != 2:
        if num == 1:
            print()
            print("네이버 파파고를 선택했습니다.")
            Papago_Vogulize(Toknized_Korean_Okt)
            break

        elif num == 2:
            print()
            print("구글 번역기를 선택했습니다.")
            Google_Vogulize(Toknized_Korean_Okt)
            break


        else:
            print("잘못 입력했습니다. 다시 선택해 주세요. (1: 네이버 파파고, 2: 구글 번역기)")
            num = int(input())

보그체로 변환하고 싶은 문장을 입력하세요.
보그체로 변환할 때 사용할 번역기의 숫자를 입력하세요. (1: 네이버 파파고, 2: 구글 번역기)

네이버 파파고를 선택했습니다.

더(We)는 심플리(simply) 사는법을 잊어버렸다.  버시(Busy) 일인허리턴스(Inheritance)에 오카시오널리(occasionally) 비치는 오아시스(Oasis) 앞에 앉은 듯한 스틸(still) 모멘트(Moment)가 찾아와도 더(We)는 세인트(That)가 더(We) 삶의 넥스트(next) 단비찬스(by chance)의 풀필먼트(fulfillment)를 리저브(reserve)해주는 밸류블(Valuable) 모멘트(Moment)이라는 걸 알지못한다. 

우리는 단순하게 사는법을 잊어버렸다.  바쁜 일상속에 간혹 비치는 오아시스 앞에 앉은 듯한 고요한 순간이 찾아와도 우리는 그것이 우리 삶의 다음 단계로의 이행을 예비해주는 귀중한 순간이라는 걸 알지못한다.
